**Installing Tensorflow GPU Version**

In [2]:
pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 92kB/s 
     |████████████████████████████████| 3.0MB 62.0MB/s 
     |████████████████████████████████| 419kB 54.0MB/s 
  Found existing installation: tensorflow-gpu 1.14.0
    Uninstalling tensorflow-gpu-1.14.0:
      Successfully uninstalled tensorflow-gpu-1.14.0


**Getting the Dataset**

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2019-09-28 06:51:34--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 2404:6800:4003:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M  37.8MB/s    in 1.7s    

2019-09-28 06:51:37 (37.8 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



**Importing the Required Files**

In [4]:
import os
import zipfile
import tensorflow as tf
import numpy as np

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)

2.0.0-alpha0


**Unzipping the File**

In [0]:
zipfile_object = zipfile.ZipFile('cats_and_dogs_filtered.zip', mode = 'r')

In [0]:
zipfile_object.extractall()

**Specifying the Directories**

In [0]:
train_dir = './cats_and_dogs_filtered/train'
validation_dir = './cats_and_dogs_filtered/validation'

**Generating the Images**

In [0]:
train_datagen = ImageDataGenerator(rescale = 1/255.0)
validation_datagen = ImageDataGenerator(rescale = 1/255.0)

In [31]:
train_generator = train_datagen.flow_from_directory(directory = train_dir, target_size = (128, 128), class_mode = 'binary')

Found 2000 images belonging to 2 classes.


In [32]:
validation_generator = validation_datagen.flow_from_directory(directory = validation_dir, target_size = (128, 128), class_mode = 'binary')

Found 1000 images belonging to 2 classes.


**Defining the Base Model**

In [0]:
IMG_SHAPE = (128, 128, 3)
base_model = tf.keras.applications.MobileNetV2(input_shape = IMG_SHAPE, include_top = False, weights = 'imagenet')

**Freezing the Base Model**

In [0]:
base_model.trainable = False

**Analyzing the Base Model Output**

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

**Making the Prediction Layer**

In [0]:
prediction_layer = tf.keras.layers.Dense(units = 1, activation = 'sigmoid')(global_average_layer)

**Making the final Model**

In [0]:
model = tf.keras.models.Model(inputs = base_model.inputs, outputs = prediction_layer)

In [0]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

**Fitting the Image Generator**

In [39]:
model.fit_generator(generator = train_generator, epochs = 5, validation_data = validation_generator)

Epoch 1/5
63/63 [==============================] - 51s 814ms/step - loss: 0.1698 - accuracy: 0.9340 - val_loss: 0.1424 - val_accuracy: 0.9510
Epoch 2/5
63/63 [==============================] - 9s 142ms/step - loss: 0.0326 - accuracy: 0.9890 - val_loss: 0.1553 - val_accuracy: 0.9530
Epoch 3/5
63/63 [==============================] - 9s 144ms/step - loss: 0.0171 - accuracy: 0.9955 - val_loss: 0.1750 - val_accuracy: 0.9600
Epoch 4/5
63/63 [==============================] - 9s 144ms/step - loss: 0.0111 - accuracy: 0.9965 - val_loss: 0.1903 - val_accuracy: 0.9600
Epoch 5/5
63/63 [==============================] - 9s 143ms/step - loss: 0.0127 - accuracy: 0.9945 - val_loss: 0.4224 - val_accuracy: 0.9180


In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(generator = validation_generator)

In [41]:
print('The generator accuracy is {}'.format(valid_accuracy))

The generator accuracy is 0.9179999828338623


**Fine tuning the model**

In [0]:
base_model.trainable = True

In [0]:
trainable_from = 100

In [0]:
for layer in base_model.layers[:trainable_from]:
  layer.trainable = False

In [0]:
model.compile(optimizer = tf.keras.optimizers.RMSprop(lr = 0.0001), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [50]:
model.fit_generator(generator = train_generator, epochs = 5, validation_data = validation_generator)

Epoch 1/5
63/63 [==============================] - 28s 442ms/step - loss: 0.0141 - accuracy: 0.9970 - val_loss: 0.1355 - val_accuracy: 0.9700
Epoch 2/5
63/63 [==============================] - 8s 127ms/step - loss: 0.0030 - accuracy: 0.9990 - val_loss: 0.2210 - val_accuracy: 0.9600
Epoch 3/5
63/63 [==============================] - 8s 127ms/step - loss: 0.0032 - accuracy: 0.9990 - val_loss: 0.2585 - val_accuracy: 0.9600
Epoch 4/5
63/63 [==============================] - 8s 126ms/step - loss: 0.0050 - accuracy: 0.9980 - val_loss: 0.3470 - val_accuracy: 0.9440
Epoch 5/5
63/63 [==============================] - 8s 128ms/step - loss: 3.8944e-04 - accuracy: 1.0000 - val_loss: 0.2235 - val_accuracy: 0.9610


In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(generator = validation_generator)

In [53]:
print('Accuracy After Fine Tuning is {}'.format(valid_accuracy))

Accuracy After Fine Tuning is 0.9610000252723694
